<a href="https://colab.research.google.com/github/richlee-Lee/richlee-code-book/blob/main/customer_churn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import tensorflow as tf

In [ ]:
print("skiki-learn version:", sklearn.__version__)
print("Tensorflow version:", tf.__version__)

In [ ]:
file_loc = "https://storage.googleapis.com/low-code-ai-book/churn_dataset.csv"
df_raw = pd.read_csv(file_loc)
df_raw.head()

In [ ]:
df_raw.info()

In [ ]:
df_raw['TotalCharges'].describe()

In [ ]:
mask = (df_raw['TotalCharges']==' ')
df_raw[mask].head()

In [ ]:
mask = (df_raw['tenure']==0)
df_raw[mask][['tenure','TotalCharges']]

In [ ]:
df1 = df_raw.copy()
df1['TotalCharges'].replace(' ',0.0, inplace=True)
df1[mask][['tenure','TotalCharges']]

In [ ]:
df1['TotalCharges'] = df1['TotalCharges'].astype('float')
df1.info()

In [ ]:
df1.describe()

In [ ]:
df1.describe(include='all')

In [ ]:
df1.value_counts(['PhoneService','MultipleLines'])

In [ ]:
df1.groupby('Contract')['Churn'].value_counts(normalize=True).unstack('Churn').plot.bar(stacked=True)

In [ ]:
def plot_cat_dist(feature_name):
    df1.groupby(feature_name)['Churn'].value_counts(normalize=True).unstack('Churn').plot.bar(stacked=True)

In [ ]:
plot_cat_dist('PaymentMethod')

In [ ]:
df1['average_charge'] = df1['TotalCharges']/df1['tenure']
df1['diff'] = df1['MonthlyCharges'] - df1['average_charge']
df1['diff'].describe()

In [ ]:
df1['month_bin'] = pd.cut(df1['MonthlyCharges'],bins=3)
plot_cat_dist('month_bin')

In [ ]:
df2 = df1.copy()
df2 = df2.drop(columns=['gender','PhoneService','StreamingTV','StreamingMovies'])
df2.columns

In [ ]:
df2 = df2.drop(columns=['customerID','TotalCharges','average_charge','diff','month_bin'])
df2.columns

In [ ]:
df2.dtypes

In [ ]:
df2.describe(include='all')

In [ ]:
df2['InternetService'].value_counts()

In [ ]:
df_prep = df2.replace('No internet service', 'No')
df_prep[['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport']].nunique()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
numeric_columns = ['SeniorCitizen', 'tenure', 'MonthlyCharges']
category_columns = ['Partner', 'Dependents', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'Contract', 'PaperlessBilling', 'PaymentMethod','diffbuckets']

x_num = df_prep[numeric_columns]
x_cat = df_prep[category_columns]
ohe = OneHotEncoder(drop='if_binary')
x_cat_trans = ohe.fit_transform(x_cat)

In [ ]:
x_cat_trans.toarray()[0]

In [ ]:
ohe.inverse_transform(x_cat_trans.toarray())[0]

In [ ]:
X = np.concatenate((x_num.values,x_cat_trans.toarray()), axis=1)
y = df_prep['Churn'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=113)
X_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
cls = LogisticRegression()
cls.fit(X_train, y_train)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
cls.fit(X_train_scaled, y_train)

In [ ]:
cls.score(X_test, y_test)

In [ ]:
X_test_scaled = scaler.transform(X_test)
cls.score(X_test_scaled, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = cls.predict(X_test_scaled)
confusion_matrix(y_test, y_pred, labels=['Yes','No'])

In [ ]:
from sklearn.metrics import precision_score, recall_score
print("Precision:",precision_score(y_test,y_pred,labels=['Yes','No'], pos_label='Yes'))
print("Recall:",recall_score(y_test,y_pred,labels=['Yes','No'],pos_label='Yes'))